In [1]:
# import os
import pickle
import warnings
import seaborn
import pandas as pd
import autograd.numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.morphology import opening, closing, ball, dilation
import skimage.measure as skm

from autograd import grad
from skimage import io
from skimage import data
from skimage import color
from skimage.morphology import opening, ball
from scipy.ndimage import fourier_shift, zoom, gaussian_filter
from scipy.sparse import csr_matrix, hstack
from scipy.stats import pearsonr
from scipy.spatial.distance import cdist
from scipy.ndimage.measurements import center_of_mass
from sklearn.decomposition import NMF, non_negative_factorization

from tqdm.notebook import tqdm

import ipywidgets as widgets
from ipywidgets import interactive, IntSlider


In [2]:
%matplotlib inline

def imagesc(img, sz1 = 10, sz2 = 10):
    plt.figure(figsize=(sz1,sz2))
    plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, left=False, labelleft=False)
    max_val = np.percentile(img, 99.9) #make sure colors are scaled over whole image
    plt.imshow(img, vmin = 0, vmax = max_val)
    
def viewmask(img, sz1 = 10, sz2 = 15):
    plt.figure(figsize=(sz1,sz2))
    plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, left=False, labelleft=False)
    plt.imshow(np.sum(img, 2), vmin = 0)

def imagesc3D(vol):
    def f(idx):
        plt.figure(figsize=(10,10))
        plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, left=False, labelleft=False)
        max_val = np.percentile(vol, 99) #make sure colors are scaled over whole image
        plt.imshow(vol[:,:, idx], vmin = 0,vmax = max_val)
        plt.show()

    interactive_plot = interactive(f, idx=IntSlider(value=0, description='slice', max=vol.shape[-1]-1, min=0, continuous_update = False))
    output = interactive_plot.children[-1]
    output.layout.height = '650px'
    return interactive_plot

def imagesc4D(vol):
    def f(cidx, zidx):
        plt.figure(figsize=(10,10))
        plt.tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, left=False, labelleft=False)
        max_val = np.percentile(vol, 99) #make sure colors are scaled over whole image
        plt.imshow(vol[:,:, zidx,cidx], vmin = 0, vmax = max_val)
        plt.show()

    interactive_plot = interactive(f, cidx=IntSlider(value=0, description='channel', max=vol.shape[-1]-1, min=0, continuous_update = False), zidx=IntSlider(value=0, description='slice', max=vol.shape[-2]-1, min=0, continuous_update = False))
    output = interactive_plot.children[-1]
    output.layout.height = '650px'
    return interactive_plot

In [3]:
def high_pass(Y, pct = 10):
    Y_new = Y.copy()
    for idx in tqdm(range(Y.shape[3])):
        Ytemp = Y[:,:,:,idx]
        thresh = np.percentile(Ytemp[Ytemp > 0], pct)
        Ytemp[Ytemp < thresh] = thresh
        Y_new[:,:,:,idx] = Ytemp - thresh

    return Y_new

def percentile_normalize(Y, pct_low = 95, pct_high = 99):
    sf = []
    Y_new = Y.copy()
    Y_new = Y_new.astype('d')
    for idx in tqdm(range(Y.shape[3])):
        in_range = np.logical_and(Y[:,:,:,idx] > np.percentile(Y[:,:,:,idx], pct_low), Y[:,:,:,idx] < np.percentile(Y[:,:,:,idx], pct_high)) 
        if np.min(Y[in_range, idx]) <= 0:
            print('WARNING: Percentile values may be poorly scaled...')
        scale_fac = np.mean(Y[in_range, idx])
        sf.append(scale_fac)
        Y_new[:,:,:,idx] = Y[:,:,:,idx]/scale_fac
    
    Y_new = Y_new*np.median(sf)
        
    return Y_new

def background_opening(Y, size = 3):
    strel = ball(size)
    Y_new = Y.copy()
    for i in tqdm(range(Y.shape[3])):
        Y_new[:,:,:,i] = Y[:,:,:,i] - opening(Y[:,:,:,i], strel)
    
    return Y_new

In [4]:
def load_fov(filepath, ext = '_affine', raw = False):
    num_channels = 3
    bases_sequenced = [1, 2, 3, 4, 6]
    
    if raw:
        file_strs = ['ch00', 'ch01', 'ch02']
    else:
        file_strs = ['ch00', 'ch01SHIFT', 'ch02SHIFT']
    filenames = []
    for ridx in bases_sequenced:
        for fs in file_strs:
            filenames.append(filepath+'richieseq_round00'+str(ridx)+'_'+fs+ext+'.tif')
            
    files = []
    for j in tqdm(range(len(bases_sequenced)*num_channels)):
        fn = filenames[j]
        print(fn)
        files.append(io.imread(fn))
    Y = np.array(files)
    Y = np.moveaxis(Y, 1, -1)
    Y = np.moveaxis(Y, 0, -1)
    
    return Y

In [5]:
def condense_puncta_signal(input_vals):
    return np.percentile(input_vals, q = 90, axis = 0)

def barcode_match(barcode, barcode_img, num_bases):
    matches = np.dot(barcode_img, barcode.T) == num_bases
    return np.squeeze(matches)

# Basecall Puncta Mask

In [30]:
fovs = ['1_0', '2_0', '3_0', '4_0', '5_0', '2_1', '3_1', '4_1', '5_1']

In [ ]:
weights = pickle.load(open('full_weights.p', 'rb'))
rweights = np.reshape(weights, (4,3))

cell_barcode = np.loadtxt('/mp/nas2/axue/fullstitch_barcode.csv', delimiter=",")
cell_cvec = np.loadtxt('/mp/nas2/axue/fullstitch_Cvec.csv', delimiter=",")

full_stitch_labels = pickle.load(open("full_stitch_label.p", "rb" ))
unique_labels = np.unique(full_stitch_labels[full_stitch_labels > 0]).astype('int')

for fov in fovs:

    Y = load_fov('/mp/nas2/DG/iarpa_virtual_tiles/' + fov + '/4_registration/')
    Y = high_pass(Y)
    Y = percentile_normalize(Y)
    
    mask = np.moveaxis(io.imread('/mp/nas2/DG/iarpa_virtual_tiles/' + fov + '/5_puncta-extraction/richieseq_allsummedSummedNorm_puncta.tif'), 0, -1)
    
    conncomp = skm.label(mask, connectivity = 1)
    properties = skm.regionprops(conncomp)
    
    #collect puncta signal
    fullC = []
    for i in tqdm(range(len(properties))):
        coords = properties[i].coords
        xc = coords[:, 0]
        yc = coords[:, 1]
        zc = coords[:, 2]
        test = Y[xc, yc, zc, :]
        cvec = condense_puncta_signal(test)
        fullC.append(cvec)

    fullC = np.array(fullC)
    
    #basecall puncta
    b1, bT = fullC.shape
    barcodeC = np.reshape(fullC, (b1, 5, 3))
    barcodeC = np.matmul(barcodeC, rweights.T)
    barcodeC = (barcodeC / np.max(barcodeC, -1, keepdims = True) == 1).astype('int')
    barcodeC = np.reshape(barcodeC, (b1,-1))
    
    label_colors = color.label2rgb(unique_labels)
    label_mask = np.zeros(Y.shape[0:3], dtype = 'uint16')

    match_dict = {}
    bidxs = []
    for tidx in unique_labels:
        bidx = np.where(unique_labels == tidx)[0][0]
        bidxs.append(bidx)
        matches = barcode_match(cell_barcode[bidx, :], barcodeC, 5)
        match_dict[tidx] = list(np.where(matches)[0])

        for puncta in tqdm(match_dict[tidx]):
            coords = properties[puncta].coords
            xc = coords[:, 0]
            yc = coords[:, 1]
            zc = coords[:, 2]

            label_mask[xc, yc, zc] = tidx
            
    pickle.dump(label_mask, open(fov+'_labelmask_51.p', 'wb'))
    pickle.dump(fullC, open(fov+'_fullc_punctamask.p', 'wb'))

/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round001_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round001_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round001_ch02SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round002_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round002_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round002_ch02SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round003_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round003_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round003_ch02SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round004_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/2_1/4_registration/richieseq_round004_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_v

<ipython-input-28-0d74c2f99c26>:38: RuntimeWarning: invalid value encountered in true_divide
  barcodeC = (barcodeC / np.max(barcodeC, -1, keepdims = True) == 1).astype('int')
<ipython-input-28-0d74c2f99c26>:41: FutureWarning: The new recommended value for bg_label is 0. Until version 0.19, the default bg_label value is -1. From version 0.19, the bg_label default value will be 0. To avoid this warning, please explicitly set bg_label value.
  label_colors = color.label2rgb(unique_labels)


/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round001_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round001_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round001_ch02SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round002_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round002_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round002_ch02SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round003_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round003_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round003_ch02SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round004_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/3_1/4_registration/richieseq_round004_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_v

/mp/nas2/DG/iarpa_virtual_tiles/4_1/4_registration/richieseq_round001_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/4_1/4_registration/richieseq_round001_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/4_1/4_registration/richieseq_round001_ch02SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/4_1/4_registration/richieseq_round002_ch00_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/4_1/4_registration/richieseq_round002_ch01SHIFT_affine.tif
/mp/nas2/DG/iarpa_virtual_tiles/4_1/4_registration/richieseq_round002_ch02SHIFT_affine.tif


# Stitch Puncta Basecalls

In [31]:
puncta_labels = {}
for fov in tqdm(fovs):
    label_mask = pickle.load(open(fov+'_labelmask_51.p', 'rb'))
    puncta_labels[fov] = label_mask

In [34]:
o_d1, o_d2, o_d3 = 1870, 1946, 400
full_labels = np.zeros((o_d1*7, o_d2*2, o_d3), dtype = 'uint16')

full_labels[1*o_d1:(1+1)*o_d1, 0*o_d2:(0+1)*o_d2,:] = puncta_labels['1_0']
full_labels[2*o_d1:(2+1)*o_d1, 0*o_d2:(0+1)*o_d2,:] = puncta_labels['2_0']
full_labels[3*o_d1:(3+1)*o_d1, 0*o_d2:(0+1)*o_d2,:] = puncta_labels['3_0']
full_labels[4*o_d1:(4+1)*o_d1, 0*o_d2:(0+1)*o_d2,:] = puncta_labels['4_0']
full_labels[5*o_d1:(5+1)*o_d1, 0*o_d2:(0+1)*o_d2,:] = puncta_labels['5_0']

full_labels[2*o_d1:(2+1)*o_d1, 1*o_d2:(1+1)*o_d2,:] = puncta_labels['2_1']
full_labels[3*o_d1:(3+1)*o_d1, 1*o_d2:(1+1)*o_d2,:] = puncta_labels['3_1']
full_labels[4*o_d1:(4+1)*o_d1, 1*o_d2:(1+1)*o_d2,:] = puncta_labels['4_1']
full_labels[5*o_d1:(5+1)*o_d1, 1*o_d2:(1+1)*o_d2,:] = puncta_labels['5_1']

In [36]:
np.unique(labels2)

array([  0,  35,  40,  44,  79,  91,  93,  99, 100, 109, 118, 205, 206,
       215, 255, 270, 277, 283, 287, 293, 294, 296, 324, 325, 328, 335,
       338, 341, 348, 375, 382, 388, 393, 401, 407, 411, 430, 435, 464,
       475, 476, 509, 512, 513, 533, 548, 583, 586, 589, 594],
      dtype=uint16)

In [53]:
cell_barcode[44,:]

array([0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0.])

In [51]:
np.where(unique_labels == 513)[0][0]

44

In [1]:
labels2 = np.max(full_labels == 35, 2)
col_labels = color.label2rgb(labels2, bg_label = 0)
plt.figure(figsize=(10,35))
plt.imshow(col_labels)

NameError: name 'np' is not defined

In [54]:
io.imsave('/mp/nas2/axue/single_puncta_labels.tif', full_labels)

<ipython-input-54-9550621390f1>:1: UserWarning: /mp/nas2/axue/single_puncta_labels.tif is a low contrast image
  io.imsave('/mp/nas2/axue/single_puncta_labels.tif', full_labels)
